# Does it get warmer before it rains?

#### More generally, does it get warmer before a precipitation event? Speculating about the weather has been a staple of small talk and human curiosity since the inception of the human race, and as a result, many weather "myths" exist.

First, let's import data and necessary packages.

In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot_date
import seaborn as sns

import datetime as dt
from datetime import datetime
from IPython.display import display

# Make a date parsing function with the correct format
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M')

# Read the CSV while parsing the dates
df = pd.read_csv("913023.csv", parse_dates=['DATE'], date_parser=dateparse)

/home/william/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8,10,11,12,13,14,15,16,17,19,20,22,23,25,26,27,28,29,33,34,37,38,39,40,44,45,46,47,48,50,59,60,62,63,64,67,68,69,71,72,73,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Remove all columns except for STATION, DATE, DAILYAverageDryBulbTemp, DAILYPrecip

stations = {'seattle': 'WBAN:24233', 
            'boston': 'WBAN:14739', 
            'lexington': 'WBAN:93820', 
            'san diego': 'WBAN:93107'}

#Make a copy of the original DataFrame
seatac_2015 = df.copy(deep=True)

#Select rows of data for Seattle (WBAN:24233)
seatac_2015 = df.loc[df['STATION'] == stations['seattle']]

#Select columns with data we need
seatac_2015 = seatac_2015[['STATION', 'DATE', 'DAILYAverageDryBulbTemp', 'DAILYPrecip']]